# 语言模型

学习目标
- 学习语言模型，以及如何训练一个语言模型
- 学习torchtext的基本使用方法 （可以通过 pip install torchtext 安装 torchtext）
    - 构建 vocabulary
    - word to index 和 index to word
- 学习torch.nn的一些基本模型
    - Linear
    - RNN
    - LSTM
    - GRU
- RNN的训练技巧
    - Gradient Clipping（梯度截取，防止题都爆炸）
- 如何保存和读取模型

我们会使用 [torchtext](https://github.com/pytorch/text) 来创建vocabulary, 然后把数据读成batch的格式。请大家自行阅读README来学习torchtext。

In [1]:
import torchtext
import torch
from torchtext.vocab import Vectors
import numpy as np
import random

USE_CUDA = torch.cuda.is_available()

# 将所有的随机数限制成相同的随机数
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)
    
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 100
MAX_VOCAB_SIZE = 50000
device = torch.device('cuda' if USE_CUDA else 'cpu')

- 我们会继续使用上次的text8作为我们的训练，验证和测试数据
- torchtext提供了LanguageModelingDataset这个class来帮助我们处理语言模型数据集
- TorchText的一个重要的概念是 Field ，塔决定了你的数据会如何被处理。我们使用 TEXT 这个field来处理文本数据。我们的TEXT field 有 lower = True 这个参数，所以所有的单词都会被lowercase。
- build_vocab 可以根据我们提供的训练数据集来创建最高频单词的单词表， max_size 帮助我们限定单词总量。
- BPTTIterator可以连续地得到连贯的句子


In [2]:
TEXT = torchtext.data.Field(lower = True)
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(path='.', 
                                                  train="text8.train.txt", 
                                                  validation='text8.dev.txt', 
                                                  test='text8.test.txt',
                                                 text_field = TEXT)
#获取训练集 验证集 测试集 数据。

In [3]:
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)

In [4]:
#len(TEXT.vocab)
#TEXT.vocab.itos[:10] # itos 将 vocab 进行 index to string 
#TEXT.vocab.stoi['apple'] # stoi 将 vocab 进行 string to index, 传入 单词进去 返回单词的 index
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits((train, val, test),
                                                                     batch_size=BATCH_SIZE,
                                                                     device = device,
                                                                     bptt_len=50,
                                                                     repeat = False,
                                                                     shuffle = True)
# bptt_len : back propagation through time lengh ,可以随意定义

In [5]:
it = iter(train_iter)
batch = next(it) # 获取一个 iterator 的数据

In [6]:
#print(batch)
#[torchtext.data.batch.Batch of size 32]
#[.text]:[torch.LongTensor of size 50x32]
#[.target]:[torch.LongTensor of size 50x32]
print(' '.join(TEXT.vocab.itos[i] for i in batch.text[:,0].data.cpu())) #打印 一个数据的 50 个单词
print(' '.join(TEXT.vocab.itos[i] for i in batch.target[:,0].data.cpu())) # 打印 target 数据

anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans <unk> of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the
originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans <unk> of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization


In [7]:
# for j in range(10):
#     batch = next(it)
#     print(j)
#     print(' '.join(TEXT.vocab.itos[i] for i in batch.text[:,0].data.cpu())) #打印 一个数据的 50 个单词
#     print(' '.join(TEXT.vocab.itos[i] for i in batch.target[:,0].data.cpu())) # 打印 target 数据
#     每个batch 有 32 组数据，每个batch 的同一组数据是连接在一起的，但不同组数据之间没有前后顺序关系。

- 此时 TEXT.vocab是有 50002 个单词的，多出来的两个特殊的token 是 \<UNK>未知的单词 \<PAD> padding

模型的输入是一串文字，模型的输出也是一串文字，他们之间相差一个位置，因为语言模型的目标是根据之前的单词预测下一个单词。

### 定义模型

- 继承nn.Module
- 初始化函数
- forward函数
- 其余可以根据模型需要定义相关的函数

In [8]:
import torch.nn as nn

class RNNModel(nn.Module):
    ''' 简单的循环神经网络 '''
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        self.linear = nn.Linear(hidden_size, vocab_size)# LSTM 的输出要映射到单词表 所以要设置 decoder
        self.hidden_size = hidden_size
        
    def forward(self, text, hidden):
        # text: seq_length * batch_size
        # hidden: (h_0,c_0) 
        # h_0: (num_layers * num_directions)* batch* hidden_size -> 1 * batch_size *hidden_seze
        # c_0: (num_layers * num_directions)* batch* hidden_size -> 1 * batch_size *hidden_seze
        
        # forward pass 
        embed = self.embed(text) # seq_length * batch_size * embed_Size
        
        output, hidden = self.lstm(embed, hidden)
        # output: seq_length * batch_size * hidden_size
        # hidden: (h_n,c_n)
        # h_n: (num_layers * num_directions)* batch* hidden_size -> 1 * batch_size *hidden_seze
        # c_n: (num_layers * num_directions)* batch* hidden_size -> 1 * batch_size *hidden_seze
        
        ''' 将 output 压缩成 2维的数据进行 linear 处理，处理完以后再解压'''
        out_vocab = self.linear(output.view(-1,output.shape[2])) #(seq_length * batch_size) * hidden_size
        out_vocab = out_vocab.view(output.size(0), output.size(1), out_vocab.size(-1)) # # seq_length * batch_size * vocab_size
        
        return out_vocab, hidden
    
    def init_hidden(self, batch_size, requires_grad=True):
        ''' 设置 weight 的技巧，因为我们不知道 weight 现在在 GPU 还是 CPU 上运行的所以使用以下方式初始化 weight '''
        weight = next(self.parameters())
        return (weight.new_zeros([1, batch_size, self.hidden_size],requires_grad=requires_grad),
               weight.new_zeros([1, batch_size, self.hidden_size],requires_grad=requires_grad)) # 1: num_layers * num_directions
        

初始化一个模型

In [9]:
model = RNNModel(vocab_size=len(TEXT.vocab),
                 embed_size=EMBEDDING_SIZE,
                 hidden_size=HIDDEN_SIZE)
if USE_CUDA:
    model = model.to(device)

In [10]:
next(model.parameters())

Parameter containing:
tensor([[-0.4862, -0.1841,  1.3934,  ..., -1.9593, -0.3455, -1.2341],
        [ 0.3047, -0.1058,  0.7007,  ..., -0.2304, -0.2342, -1.8808],
        [-0.6481,  1.1889,  1.2151,  ..., -0.0274, -0.3379,  1.5340],
        ...,
        [ 1.3092,  0.9815, -0.8608,  ..., -0.5230,  0.1672,  2.0918],
        [-1.4777, -1.3770, -1.0617,  ...,  0.8354,  0.9008,  0.5809],
        [ 1.3996, -0.6499,  1.0205,  ..., -1.1576, -1.0426,  0.5921]],
       requires_grad=True)

- 我们首先定义评估模型的代码。
- 模型的评估和模型的训练逻辑基本相同，唯一的区别是我们只需要forward pass，不需要backward pass

我们需要定义下面的一个function，帮助我们把一个hidden state和计算图之前的历史分离。

定义loss function和optimizer

训练模型：
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- gradient clipping，防止梯度爆炸
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证集上做模型的评估

In [11]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor): # isinstance 判断 h 是否是 torch.Tensor 类型的 ，也可以用 type（h）= = torch.Tensor 代替
        ''' detach：返回一个新的从当前计算图中分离出来的 variable，当计算反向传播梯度时，就会在当前的 variable 停止不会再往前传播
            这里使用detach是因为，batch之间的数据是连续的，我们想保存之前计算出来的结果，但在计算反向传播梯度时，我们只想计算当前
            batch的梯度不想计算前面batch的梯度，所以使用 detach 将计算结果剥离出来，也可以使用将计算结果转换为数值，然后重新定义
            新的tensor的方式.
            detach 时，梯度会清零
        '''
        return h.detach() 
    else:
        ''' tuple 元组，内部元素以逗号分隔 '''
        return tuple(repackage_hidden(v) for v in h)

In [24]:
def evaluate(model,data):
    model.eval()# 模型切换至验证模式
    total_loss = 0.
    total_count = 0.
    BATCH_SZIE = 32
    it = iter(data)
    with torch.no_grad(): # 因为是验证所以不需要计算梯度
        hidden = model.init_hidden(BATCH_SZIE,requires_grad=False)
        for i,batch in enumerate(it):
            data, target = batch.text, batch.target

            ''' 因为此时的 hidden 是一个tensor 保存了过往所有的记录，随着时间的累计内存被占用越来越多，
                我们只希望保存tensor里的数据，不希望保存tensor的历史记录
                所以设置了 repackage_hidden 方法，去重置 hidden'''
            hidden = repackage_hidden(hidden)
            output,hidden = model(data,hidden) 
            ''' view(-1,vocab_size) 表示 将tensor转换为两维，且第一维的个数由第二维度的个数来判断'''
            loss = loss_fn(output.view(-1,VOCAB_SIZE),target.view(-1)) # cross entorpy loss 传进来的是 batch_size * target_class_dim , batch_size 
            ''' 因为 loss 在计算时取平均，所以我们希望能把这些loss 加回来，
                np.multiply(*data.size()) ：data.size 是一个 tuple ，multiply 是将 tuple 内的数据相乘计算出数据的个数。
            '''
            total_loss += loss.item()*np.multiply(*data.size())
            total_count += np.multiply(*data.size())
    
    loss = total_loss / total_count
    model.train()# 模型切换回 train 模式
    return loss
            

In [ ]:
loss_fn = nn.CrossEntropyLoss()
LEARNING_RATE = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5) # 对学习率进行微调，每次调整为之前的一半
VOCAB_SIZE = len(TEXT.vocab)

In [ ]:
NUM_EPOCHS = 2
GRAD_CLIP = 5.0
val_losses = []
for epoch in range(NUM_EPOCHS):
    model.train() # pytorch 中 model 有多种模式，此时使用的是 train 模式
    it = iter(train_iter)
    hidden = model.init_hidden(BATCH_SIZE)
    for i,batch in enumerate(it):
        data, target = batch.text, batch.target
        
        ''' 因为此时的 hidden 是一个tensor 保存了过往所有的记录，随着时间的累计内存被占用越来越多，
            我们只希望保存tensor里的数据，不希望保存tensor的历史记录
            所以设置了 repackage_hidden 方法，去重置 hidden'''
        hidden = repackage_hidden(hidden)
        output,hidden = model(data,hidden) 
        
        optimizer.zero_grad()
        ''' view(-1,vocab_size) 表示 将tensor转换为两维，且第一维的个数由第二维度的个数来判断'''
        loss = loss_fn(output.view(-1,VOCAB_SIZE),target.view(-1)) # cross entorpy loss 传进来的是 batch_size * target_class_dim , batch_size 
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),GRAD_CLIP) # 梯度截取，防止梯度爆炸
        optimizer.step()
        
        if i % 100 ==0:
            print('epoch: ',epoch,' iterator: ',i,' loss: ',loss.item())
            
        ''' 模型的保存
            首先使用验证集的数据对模型进行验证，如果验证集的 loss 比之前的 loss 低，说明模型有进步，保存模型，
            如果验证集的 loss 没有降低，说明模型可能收敛了，需要进一步调整 learning rate ，
            learning rate 的 调整是使用 scheduler 进行调整的。 torch.optim.lr_scheduler.ExponentialLR( optimizer, 0.5)
        '''
        if i%1000 ==0:
            val_loss = evaluate(model, val_iter)
            print('epoch: ',epoch,' iterator: ',i,' loss: ',val_loss.item())
            if len(val_losses)==0 or val_loss< min(val_losses):
                torch.save(model.state_dict(), 'LanguageModel.pth')
                print(' best model saved to LanguageModel.pth')
            else:
                print('learning rate decay')
                scheduler.step()
            val_losses.append(val_loss)

加载模型

In [ ]:
best_model = RNNModel(vocab_size=len(TEXT.vocab),
                 embed_size=EMBEDDING_SIZE,
                 hidden_size=HIDDEN_SIZE)
if USE_CUDA:
    best_model = best_model.to(device)
best_model.load_state_dict(torch.load('LanguageModel.pth'))

### 使用最好的模型在valid数据上计算perplexity

In [ ]:
val_loss = evaluate(best_model,val_iter)
print('perplexity: ',np.exp(val_loss))

### 使用最好的模型在测试数据上计算perplexity

In [ ]:
test_loss = evaluate(best_model,test_iter)
print('perplexity: ',np.exp(test_loss))

使用训练好的模型生成一些句子。

In [ ]:
hidden = best_model.init_hidden(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input = torch.randint(VOCAB_SIZE, (1, 1), dtype=torch.long).to(device)
words = []
for i in range(100):
    output, hidden = best_model(input, hidden)
    word_weights = output.squeeze().exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0] # 根据output的结果的概率进行采样，也可以直接选取 argmax 的index
    input.fill_(word_idx) # 更新 input
    word = TEXT.vocab.itos[word_idx]
    words.append(word)
print(" ".join(words))